In [ ]:
#Import our necessary python libraries
import pandas as pd
import requests
import json
import os
from collections import defaultdict
from IPython.display import HTML
import plotly.express as px
BLS_API_KEY  = 'd8644c9411c54f5d8333afc4185af8e1'
BLS_ENDPOINT = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

In [ ]:
#Grab our data from covidtracking.com through their API.
df_states_current = pd.read_json("https://covidtracking.com/api/states")
df_states_daily   = pd.read_json("https://covidtracking.com/api/us/daily")
df_in_the_press   = pd.read_json("https://covidtracking.com/api/press")
df_us_current     = pd.read_json("https://covidtracking.com/api/us")
df_us_daily       = pd.read_json("https://covidtracking.com/api/us/daily")
df_counties       = pd.read_json("https://covidtracking.com/api/counties")


In [ ]:
def get_bls_data(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"seriesid": series,'registrationKey': 'd8644c9411c54f5d8333afc4185af8e1', "startyear":"%d" % (start), "endyear":"%d" % (end)})
    p = requests.post(BLS_ENDPOINT, data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])


start = 2019
end = 2020

In [ ]:

# series = ['LNS14000000', 'LNU05026645', 'LNS12032194', 'LNS14027689']
# unemp = ['LASST340000000000003','LASST340000000000004','LASST340000000000005','LASST340000000000006','LASST340000000000007','LASST340000000000008']
unemp = ['LASST340000000000004']

df_unemployment = get_bls_data(series=unemp, start=start, end=end)

In [ ]:
df_unemployment

In [ ]:
df_us_daily[["date","total"]]


In [13]:
#Choropleth Map of positive COVID-19 Cases in the United States
statesDF = df_states_current[["state","positive"]]
fig = px.choropleth(locations=statesDF.state, locationmode="USA-states", color=statesDF.positive, scope="usa",
                   labels={'color':'Positive COVID-19 Cases'})
fig.update_layout(title='COVID-19 Virus Positive Cases By State (Reported)')
fig.show()

In [ ]:
df_counties


In [ ]:

# df_us_daily['date'] = df_us_daily['date'].astype('datetime64[ns]')
df_us_daily

In [17]:
df_states_daily['dateChecked'] = df_states_daily['dateChecked'].astype('datetime64[ns]')
df_states_daily

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,death,hospitalized,total,totalTestResults,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200503,56,1152006,5901360.0,2812.0,50312.0,125796.0,9292.0,4502.0,4761.0,...,61868.0,125796.0,7056178,7053366,7053366,1158.0,1936.0,210732.0,26287.0,237019.0
1,20200502,56,1125719,5690628.0,1578.0,51734.0,123860.0,9403.0,4386.0,4845.0,...,60710.0,123860.0,6817925,6816347,6816347,1651.0,2112.0,234499.0,30038.0,264537.0
2,20200501,56,1095681,5456129.0,1639.0,52370.0,121748.0,9205.0,4300.0,4707.0,...,59059.0,121748.0,6553449,6551810,6551810,1743.0,9793.0,271571.0,33547.0,305118.0
3,20200430,56,1062134,5184558.0,2775.0,53793.0,111955.0,9486.0,4192.0,4712.0,...,57316.0,111955.0,6249467,6246692,6246692,2091.0,2265.0,191545.0,28977.0,220522.0
4,20200429,56,1033157,4993013.0,4832.0,54930.0,109690.0,9612.0,4093.0,4796.0,...,55225.0,109690.0,6031002,6026170,6026170,2700.0,3382.0,202877.0,27565.0,230442.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20200126,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,0.0,0.0,0.0,0.0,0.0
99,20200125,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,0.0,0.0,0.0,0.0,0.0
100,20200124,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,0.0,0.0,0.0,0.0,0.0
101,20200123,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,0.0,0.0,0.0,0.0,0.0


In [18]:
fig = px.line(df_us_daily, x='date', y='total')
fig.show()

In [22]:
min_date = (df_states_daily.dateChecked.min()).date()
max_date = (df_states_daily.dateChecked.max()).date()

In [23]:
tick_LH = yf.Ticker("LH")
df_LH = tick_LH.history(
    start=min_date,
    end=max_date)

In [24]:
tick_RHHBY = yf.Ticker("RHHBY")
df_RHHBY = tick_RHHBY.history(
    start=min_date,
    end=max_date)

In [25]:
tick_HOLX = yf.Ticker("HOLX")
df_HOLX = tick_HOLX.history(
    start=min_date,
    end=max_date)

In [26]:
tick_RHHBY = yf.Ticker("RHHBY")
df_RHHBY = tick_RHHBY.history(
    start=min_date,
    end=max_date)

In [27]:
tick_TMO = yf.Ticker("TMO")
df_TMO = tick_TMO.history(
    start=min_date,
    end=max_date)

In [28]:
tick_ABT = yf.Ticker("ABT")
df_ABT = tick_ABT.history(
    start=min_date,
    end=max_date)

In [33]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Positive COVID-19 cases
# fig.add_trace(go.Scatter(x=df_states_daily['dateChecked'], y=df_states_daily['positive'],
#                     mode='lines',
#                     name='Positive'),
#              secondary_y=False)

# Total tested cases
fig.add_trace(go.Scatter(x=df_states_daily['dateChecked'], y=df_states_daily['totalTestResults'],
                    mode='lines',
                    name='Tested'),
             secondary_y=False,)

# MSFT Volumne
fig.add_trace(go.Scatter(x=df_HOLX.index, y=df_HOLX['Close'],
                    mode='lines',
                    name='$HOLX'),
             secondary_y=True,)



fig.add_trace(go.Scatter(x=df_RHHBY.index, y=df_RHHBY['Close'],
                    mode='lines',
                    name='$RHHBY'),
             secondary_y=True,)

# fig.add_trace(go.Scatter(x=df_TMO.index, y=df_TMO['Close'],
#                     mode='lines',
#                     name='$TMO'),
#              secondary_y=True,)

# fig.add_trace(go.Scatter(x=df_LH.index, y=df_LH['Close'],
#                     mode='lines',
#                     name='$LH'),
#              secondary_y=True,)
# fig.add_trace(go.Scatter(x=df_ABT.index, y=df_ABT['Close'],
#                     mode='lines',
#                     name='$ABT'),
#              secondary_y=True,)


fig.update_layout(
    title_text="Closing Values during CoViD-19"
)

fig.update_layout(
    showlegend=False,
    annotations=[
        dict(
            x=2,
            y=5,
            xref="x",
            yref="y",
            text="dict Text",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        )
    ]
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Cases", secondary_y=False)
fig.update_yaxes(title_text="Price", secondary_y=True)
fig.update_xaxes(rangeslider_visible=True)



fig.show()

In [30]:

# Create a list of annotations
# annotation = [dict(x=df_states_daily['dateChecked'], y = value, 
#            xref = 'x', yref = 'y', 
#            font = dict(color = 'blue'),
#            text = "March 23'rd Market Crash")]

fighigh = make_subplots(specs=[[{"secondary_y": True}]])

# Positive COVID-19 cases
# fig.add_trace(go.Scatter(x=df_states_daily['dateChecked'], y=df_states_daily['positive'],
#                     mode='lines',
#                     name='Positive'),
#              secondary_y=False)

# Total tested cases
fighigh.add_trace(go.Scatter(x=df_states_daily['dateChecked'], y=df_states_daily['totalTestResults'],
                    mode='lines',
                    name='Tested'),
             secondary_y=False,)

# fighigh.add_trace(go.Scatter(x=df_HOLX.index, y=df_HOLX['Close'],
#                     mode='lines',
#                     name='$HOLX'),
#              secondary_y=True,)



# fighigh.add_trace(go.Scatter(x=df_RHHBY.index, y=df_RHHBY['Close'],
#                     mode='lines',
#                     name='$RHHBY'),
#              secondary_y=True,)

fighigh.add_trace(go.Scatter(x=df_TMO.index, y=df_TMO['Close'],
                    mode='lines',
                    name='$TMO'),
             secondary_y=True,)

# fighigh.add_trace(go.Scatter(x=df_LH.index, y=df_LH['Close'],
#                     mode='lines',
#                     name='$LH'),
#              secondary_y=True,)
# fighigh.add_trace(go.Scatter(x=df_ABT.index, y=df_ABT['Close'],
#                     mode='lines',
#                     name='$ABT'),
#              secondary_y=True,)


# Add figure title
fighigh.update_layout(
    title_text="Closing Values during CoViD-19"
)

# Set x-axis title
fighigh.update_xaxes(title_text="Date")

# Set y-axes titles
fighigh.update_yaxes(title_text="Cases", secondary_y=False)
fighigh.update_yaxes(title_text="Price", secondary_y=True)
fighigh.update_xaxes(rangeslider_visible=True)



fighigh.show()

In [ ]:
df_states_daily['dateChecked'] = pd.to_datetime(df_states_daily['dateChecked'])

In [ ]:
marchcrash_start = '20200323'
marchcrash_end = '20200324'
df_states_daily['date'] = df_states_daily.date.to_datetime(df_states_daily['date'])
after_start_date = df_states_daily['date'] >= marchcrash_start
before_end_date = df_states_daily['date'] <= marchcrash_end
between_two_dates = after_start_date & before_end_date
filtered_dates = df_states_daily.loc[between_two_dates]
# crash = df_states_daily['dateChecked'] == marchcrash

In [ ]:
df_states_daily['date']

In [ ]:
crash_df = df_states_daily.loc[crash]

In [ ]:
crash_df